In [1]:
import torch
from concrete.ml.torch.compile import compile_brevitas_qat_model
from mingpt.model import GPT
from mingpt.utils import CfgNode as CN

/Users/davidhu/Documents/code/ml/65610_project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/davidhu/Documents/code/ml/65610_project/venv/lib/python3.10/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
from fhegpt.qmodel import QuantCasualSelfAttention

In [3]:
config = CN(
    n_embd = 768,
    n_layer = 12,
    n_head=12,
    attn_pdrop=0.1,
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    block_size=1000
)
qcsa = QuantCasualSelfAttention(config)

In [4]:
print(qcsa)

QuantCasualSelfAttention(
  (quant_inp): QuantIdentity(
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (act_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (fused_activation_quant_proxy): FusedActivationQuantProxy(
        (activation_impl): Identity()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClamp()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
          )
          (scaling_impl): ParameterFromRuntimeStatsScaling(
            (stats_input_view_shape_impl): OverTensorView()
            (stats): _Stats(
              (stats_impl): AbsPercentile()
            )
            (restrict_scaling): _RestrictValue(
              (restrict_value_impl): FloatRestrictValue()
            )
            (clamp_scaling): _ClampValue(
           

In [5]:
torch_input = torch.randn(1, 10, config.n_embd)
quantized_model = compile_brevitas_qat_model(qcsa, torch_input)

x size torch.Size([1, 10, 768])
k size torch.Size([1, 12, 10, 64])
att size torch.Size([1, 12, 10, 10])
type:<built-in method type of Tensor object at 0x14f3f6840>
tensor([[[[ 2.0398e-01,  1.7629e-02, -6.8057e-02,  ...,  7.7710e-02,
           -1.8742e-01,  1.4398e-02],
          [ 1.3464e-01,  2.3380e-02,  3.1497e-01,  ...,  5.3294e-02,
           -1.7721e-01,  2.2448e-01],
          [-5.8807e-02, -9.4393e-02, -9.4535e-02,  ..., -2.1856e-02,
            1.1519e-01, -1.1417e-01],
          ...,
          [ 3.5671e-02, -1.8994e-01,  2.7274e-02,  ...,  1.8284e-01,
            6.8152e-02,  3.6103e-02],
          [-6.8179e-02,  5.0569e-02, -1.6007e-02,  ..., -8.6557e-02,
            4.5674e-02, -1.9597e-01],
          [ 1.8493e-03, -3.9756e-02,  2.7428e-01,  ...,  1.4109e-01,
            4.3118e-02, -1.6142e-01]],

         [[ 3.1261e-01,  2.3545e-01,  1.3841e-01,  ...,  1.3233e-01,
           -2.0676e-01,  6.2819e-02],
          [-1.5730e-01, -1.3279e-01,  2.4011e-01,  ...,  5.0064e-02,
 

/Users/davidhu/Documents/code/ml/65610_project/concrete-ml/src/concrete/ml/quantization/quantizers.py:497: RuntimeWarning: invalid value encountered in float_scalars
  if stats.rmax - stats.rmin < STABILITY_CONST:
/Users/davidhu/Documents/code/ml/65610_project/concrete-ml/src/concrete/ml/onnx/ops_impl.py:149: RuntimeWarning: invalid value encountered in multiply
  return c * t + (1.0 - c) * f


AssertionError: QuantizedOps whose ONNX numpy implementation is marked as producing raw output must return an instance of RawOpOutput. 
 ** Offending Op: ONNXShape

# 